# Data visualization throttled:
Unthrottled memory power = ~3000

| # | Setting |
| --- | --- |
| Device | macbook pro 14-inch, 2023 | 
| Chip | Apple M2 Pro | 
| Memory | 16 GB | 
| macOS | 15.1.1 | 
| Java | openjdk 21.0.5 | 
| Node | v23.3.0 | 
| Angular | v18.2.x | 

## Throttling

| # | Setting |
| --- | --- |
| CPU | 4x CPU slowdown | 
| download | 1.6 Mbps | 
| upload | 750 Kbps | 
| latency | 150ms | 

## Core Web Vitals Measurements:

**Long task**
- Task that takes more than 50ms

**TTFB (Time to First Byte):**
 - Time from navigation start until the first byte of response is received
 - Calculated using: performance.getEntriesByType('navigation')[0].responseStart
 
**FCP (First Contentful Paint):**
 - Time when the first text, image, or other content appears on the screen.
 - Is the 'endTime' of the first-contentful-paint
 - FCP includes all things that happened before it (like TTFB).
 
**LCP (Largest Contentful Paint):**
 - Time when the largest text or image element is rendered on the screen (so endTime)
 - Is the 'endTime' of the largest-contentful-paint.
 - FCP includes all things that happened before it (like TTFB).
 
**TTI (Time to Interactive):**
 - Time when the page becomes 'reliably' interactive
 - It is measured as the endtime of the last 'long task' after the FCP starttime
 - If no tasks are found, it is the end-time of the FCP.
 
**TBT (Total Blocking Time):**
 - Sum of all "blocking time" for long tasks between start of FCP and TTI
 - Blocking time = accumulation of (task duration - 50ms) of each long task.
 - TBT contains a 'quiet window' of 5s, if this window is expired, the test will end.
 - The quiet window starts after the FCP. 

In [ ]:
run6path = '../../../data/core-web-vitals/testrun-6'
run7path = '../../../data/core-web-vitals/testrun-7'

In [ ]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests

pd.set_option('display.float_format', lambda x: '%.2f' % x)


dataset_A = pd.read_csv(f'{run7path}/results-csr.csv', sep=',')
dataset_B = pd.read_csv(f'{run7path}/results-ssrh.csv', sep=',')

features = ['navTime', 'totalTime', 'lcp', 'fcp', 'ttfb', 'tbt', 'tti', 'longestTask', 'longTasks']
target_features = ['lcp', 'fcp', 'ttfb', 'tbt', 'tti', 'navTime']

# Remove Java warmup
dataset_A = dataset_A.iloc[5:]
dataset_B = dataset_B.iloc[5:]

datasets =	{
  "csr": dataset_A,
  "ssrh": dataset_B,
}

csr_dataset.info()

In [ ]:

def plot_histogram(d, f, loc):
    loc.hist(datasets[d][f].dropna(), bins=30, alpha=0.7, label=d, edgecolor='k')
    loc.set_title(d)
    loc.set_ylabel('Frequency')
    loc.set_xlabel(f'Distribution of {f}')
    loc.legend()

def graphs(plot_func, features, height=8):
    rows = len(features)
    cols = len(datasets)

    _, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(20, height*rows))
    plt.subplots_adjust(hspace=0.5)
    for y, f in enumerate(features):
        for x, d in enumerate(datasets):
            plot_func(d, f, axes[y, x])
    return plt

plt = graphs(plot_histogram, target_features)
plt.show()

In [ ]:
def detect_outliers(df, features, contamination=0.1):
    clf = IsolationForest(contamination=contamination, random_state=42)
    outliers = clf.fit_predict(df[features])
    return outliers == 1

# def plot_outliers(d, f, loc, mask):
#     loc.hist(datasets[d][f][mask[d]], bins=30, alpha=0.7, label='Inliers', edgecolor='k')
#     loc.hist(datasets[d][f][~mask[d]], bins=30, alpha=0.7, label='Outliers', color='red', edgecolor='k')
#     loc.set_title(d)
#     loc.set_ylabel('Frequency')
#     loc.set_xlabel(f'Distribution of {f}')
#     loc.legend()


def plot_outliers(d, f, loc, mask):
    loc.plot(datasets[d][f], label='dirty', color='gray', alpha=0.5)
    loc.plot(datasets[d][f][mask[d]], label=d)
    loc.plot(datasets[d][f][~mask[d]], label='outliers', color='red')

    loc.set_title(d)
    loc.set_ylabel(f'{f} Time (ms)') 
    loc.set_xlabel('test nr #') 

def visualize_outliers(features, masks, height=8):
    rows = len(features)
    cols = len(datasets)
    
    _, axes = plt.subplots(nrows=rows, ncols=cols, figsize=(20, height*rows))
    plt.subplots_adjust(hspace=0.5)
    
    for y, f in enumerate(features):
        for x, d in enumerate(datasets):
            plot_outliers(d, f, axes[y, x], masks)
    return plt

def print_outlier_stats(datasets, masks):
    stats = {}
    for name, df in datasets.items():
        total = len(df)
        outliers = (~masks[name]).sum()
        pct = (outliers / total) * 100
        stats[name] = {
            'total': total,
            'outliers': outliers,
            'percentage': pct
        }
    
    stats_df = pd.DataFrame({
        'Total Points': [stats[name]['total'] for name in stats],
        'Outliers': [stats[name]['outliers'] for name in stats],
        'Percentage Removed': [f"{stats[name]['percentage']:.1f}%" for name in stats]
    }, index=datasets.keys())
    
    return stats_df

masks = {}
cleaned_datasets = {}

for name, df in datasets.items():
    mask = detect_outliers(df, target_features)
    masks[name] = mask
    cleaned_datasets[name] = df[mask].copy()

print(print_outlier_stats(datasets, masks))

visualize_outliers(target_features, masks)
plt.show()

In [ ]:
def ceildiv(a, b): return -(a // -b)

def plot_box_plot(f, loc):
    [loc.plot(cleaned_datasets[d][f], label=d) for d in cleaned_datasets]

    loc.set_title(f)
    loc.set_ylabel('Time (ms)') 
    loc.set_xlabel('test nr #') 
    loc.set_ylim(bottom=0)
    loc.legend()

def graphs(plot_func, features, height=10):
    rows = len(features)

    _, axes = plt.subplots(nrows=rows, figsize=(20, height*rows))
    plt.subplots_adjust(hspace=0.3, top=0.95, bottom=0.05)  
    for y, f in enumerate(features):
        plot_func(f, axes[y])
    return plt

graphs(plot_box_plot, target_features)

In [ ]:
metrics = {
    'ttfb': 'two-sided',
    'fcp': 'two-sided', 
    'lcp': 'two-sided',
    'tbt': 'two-sided',
    'tti': 'two-sided'
}

keys = list(cleaned_datasets.keys())

results = pd.DataFrame([
    [metric, alt, stats.mannwhitneyu(cleaned_datasets[keys[0]][metric], 
                                   cleaned_datasets[keys[1]][metric], 
                                   alternative=alt)[1]]
    for metric, alt in metrics.items()
], columns=['metric', 'hypothesis', 'p_value'])

results['adjusted_p'] = multipletests(results['p_value'], method='bonferroni')[1]
results['significant'] = results['adjusted_p'] < 0.05

compact_format = lambda x: '%.3e' % x if x < 0.00001 else '%.5f' % x
results['p_value'] = results['p_value'].apply(compact_format)
results['adjusted_p'] = results['adjusted_p'].apply(compact_format)

results

In [ ]:
def plot_bar_chart(feature, loc):
    percentiles = {d: cleaned_datasets[d][feature].quantile(0.75) for d in datasets}
    bars = loc.bar(percentiles.keys(), percentiles.values())
    
    baseline = list(percentiles.values())[0]
    for i, bar in enumerate(bars):
        height = bar.get_height()
        percentage = ((height / baseline) - 1) * 100
        label = f'100 %' if (i == 0) else f'{percentage:+.1f}%'
        loc.text(bar.get_x() + bar.get_width()/2., height,
                label,
                ha='center', va='bottom')
    
    loc.set_title(feature)
    loc.set_ylabel('Average Time (ms)')
    loc.set_xlabel('Dataset')

def graphs(plot_func, features, height=10):
    cols = len(features)
    _, axes = plt.subplots(ncols=cols, figsize=(20, 5))
    plt.subplots_adjust(hspace=0.3, wspace=0.5, top=0.95, bottom=0.05)
    for y, f in enumerate(features):
        plot_func(f, axes[y])
    return plt

graphs(plot_bar_chart, target_features)
plt.show()

In [ ]:
metric_order = ['ttfb', 'fcp', 'tti', 'lcp', 'navTime'] 
plt.figure(figsize=(12, 6))

x = np.arange(len(datasets))
bar_width = 0.15 

for i, metric in enumerate(metric_order):
    values = [dataset[metric].quantile(0.75) for name, dataset in cleaned_datasets.items()]
    position = x + (i - len(metrics)/2 + 0.5) * bar_width
    plt.bar(position, values, bar_width, label=metric.upper())

plt.xlabel('Datasets')
plt.ylabel('Time (ms)')
plt.title('Web Performance Metrics Comparison')
plt.xticks(x, datasets.keys())
plt.legend()

plt.tight_layout()
plt.show()